In [90]:
import os
import re
os.chdir('../')

In [233]:
import json
import requests
import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [10]:
cities = [
    'bengaluru',
    # 'national-capital-region-ncr',
]

In [56]:
# https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi
# language = '?languages=hindi'
# path_base_url = 'https://in.bookmyshow.com/explore/movies-'

path_base_url = 'https://paytm.com/movies/'

delay = 300 # Delays the script execution
headers = {
'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Linux",
'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

In [57]:
for city in cities:
    path_url =  path_base_url + city
    print(f'path_url: {path_url}')

path_url: https://paytm.com/movies/bengaluru


In [58]:
response = requests.get(path_url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [71]:
class_one = 'MovieCityLandingDesktop_outercontainer__WJTmM'

In [81]:
result = soup.findAll('div')
print(f'result: {len(result)}')

result: 720


In [102]:
movie_list = soup.findAll('div', class_=re.compile("moviesList"))
print(f'result: {len(movie_list)}')

result: 1


In [111]:
running_movie_list = movie_list[0].findAll('div', class_=re.compile("RunningMovie_details"))
print(f'result: {len(running_movie_list)}')

result: 36


In [125]:
movie_data = []

for index in tqdm(range(len(running_movie_list))):
    temp_running_movie = running_movie_list[index]

    temp_data = temp_running_movie.find('a', href=True)
    temp_movie_data = {
        'name': temp_data.text,
        'url': temp_data.get('href')
    }
    
    movie_data.append(temp_movie_data)



100%|██████████| 36/36 [00:00<00:00, 34356.07it/s]


In [127]:
running_movie_list[1]

<div class="MobileRunningMovie_details__oEcin"><div class="MobileRunningMovie_rowMovie__Tr9_Z"><div class="MobileRunningMovie_detMovie__YBAdj"><a class="MobileRunningMovie_lineClamp__akkn9 MobileRunningMovie_clampTwo__eTC0p" href="/movies/pathaan-movie-detail-152325">Pathaan</a><span class="MobileRunningMovie_lineClamp__akkn9 MobileRunningMovie_clampOne__0_q0w"><span>U/A</span>Hindi, Telugu</span></div><div class="MobileRunningMovie_butBook__gc8PY"><div><button class="Button_btn__NHo2w Button_is-secondary__J8lii Button_is-medium__qKDWu">Book Now</button></div></div></div><div class="MobileRunningMovie_tag__x1KWx"><span>2D</span><span>4DX-2D</span><span>IMAX 2D</span></div></div>

In [138]:
path_movie_url = 'https://in.bookmyshow.com/buytickets/pathaan-bengaluru/movie-bang-ET00323848-MT/20230203'
response = requests.get(path_movie_url, headers = headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [150]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
  

In [279]:
class BookMyShowScrap:
    def __init__(self, path_movie_url):
        self.path_movie_url = path_movie_url    

    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers
        
    def _get_soup(self):
        headers = self._get_header()

        response = requests.get(self.path_movie_url, headers = headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    
    def _get_chrome_driver(self):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(self.path_movie_url)
        return driver

    
    def convert_to_json(self, data_categories:str):
        return json.loads(data_categories)

    def get_price(self, data_json):
        return data_json.get('price')

    def get_desc(self, data_json):
        return data_json.get('desc')

    def get_availability_class(self, data_json):
        return data_json.get('availabilityClass')

    def get_availStatus(self, data_json):
        return data_json.get('availStatus')

    def get_processed_df(self):
        driver = self._get_chrome_driver()
        
        print(driver)
        theatre_element_list = driver.find_elements(By.CLASS_NAME, 'list')
        print(f'Number of theatres : {len(theatre_element_list)}')

        data = []

        for i, a in tqdm(enumerate(theatre_element_list)):
            try:
                titles = a.find_element(By.CLASS_NAME, '__title')
                listing_info = a.find_elements(By.CLASS_NAME, 'listing-info')
                showtimes = a.find_elements(By.CLASS_NAME, 'showtime-pill')

                for s in showtimes:

                    data_categories = self.convert_to_json(s.get_attribute('data-categories'))
                    timings = s.get_attribute('data-date-time')

                    temp_data = {
                        'title' : titles.text,
                        'timings': timings,
                        'latitude': a.get_attribute('data-lat'),
                        'longtitude' : a.get_attribute('data-lng'),

                        'price': self.get_price(data_categories),
                        'description': self.get_desc(data_categories),

                        'availability_class': self.get_availability_class(data_categories),
                        'avail_status': self.get_availStatus(data_categories)

                    }
                    
                    data.append(temp_data)

                # b.find_element('showtime-pill')
            except:
                print(i)

        
        data_df = pd.DataFrame(data)

        return data_df
        

In [285]:
path_url = 'https://in.bookmyshow.com/buytickets/pathaan-bengaluru/movie-bang-ET00323848-MT/20230204'
path_url = 'https://in.bookmyshow.com/buytickets/avatar-the-way-of-water-3d-bengaluru/movie-bang-ET00342441-MT/20230204'
bmys = BookMyShowScrap(path_movie_url=path_url)

In [286]:
data_df = bmys.get_processed_df()
data_df.head()

/tmp/ipykernel_93066/2141217020.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<selenium.webdriver.chrome.webdriver.WebDriver (session="3567e832308c9167275a5a599b19af62")>
Number of theatres : 33
0
1
2
3
4
5
31
32


,title,timings,latitude,longtitude,price,description,availability_class,avail_status
0,"PVR: Soul Spirit Central Mall, Bellandur\nINFO",10:05 PM,12.9259,77.6754,510.00,RECLINER,_filling,2
1,"PVR: Nexus (Formerly Forum), Koramangala\nINFO",11:00 AM,12.9346,77.6111,240.00,CLASSIC,_filling,2
2,"PVR: Nexus (Formerly Forum), Koramangala\nINFO",02:55 PM,12.9346,77.6111,300.00,CLASSIC,_filling,1
3,"PVR: Nexus (Formerly Forum), Koramangala\nINFO",09:20 PM,12.9346,77.6111,300.00,CLASSIC,_available,3
4,"PVR: Vega City, Bannerghatta Road\nINFO",11:10 AM,12.9077,77.6012,250.00,CLASSIC,_filling,2


In [278]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(path_url)

/tmp/ipykernel_93066/1840110318.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [262]:
theatre_element_list = driver.find_elements(By.CLASS_NAME, 'list')
print(f'Number of theatres : {len(theatre_element_list)}')

Number of theatres : 85


In [ ]:
def convert_to_json(data_categories:str):
    return json.loads(data_categories)

def get_price(data_json):
    return data_json.get('price')

def get_desc(data_json):
    return data_json.get('desc')

def get_availability_class(data_json):
    return data_json.get('availabilityClass')

def get_availStatus(data_json):
    return data_json.get('availStatus')


In [254]:
data = []

for i, a in enumerate(theatre_element_list):
    try:
        titles = a.find_element(By.CLASS_NAME, '__title')
        listing_info = a.find_elements(By.CLASS_NAME, 'listing-info')
        showtimes = a.find_elements(By.CLASS_NAME, 'showtime-pill')

        for s in showtimes:

            data_categories = convert_to_json(s.get_attribute('data-categories'))
            timings = s.get_attribute('data-date-time')

            temp_data = {
                'title' : titles.text,
                'timings': timings,
                'latitude': a.get_attribute('data-lat'),
                'longtitude' : a.get_attribute('data-lng'),

                'price': get_price(data_categories),
                'description': get_desc(data_categories),

                'availability_class': get_availability_class(data_categories),
                'avail_status': get_availStatus(data_categories)

            }
            
            data.append(temp_data)

        # b.find_element('showtime-pill')
    except:
        print(i)

0
1
2
3
4
5
73
74


In [255]:
data_df = pd.DataFrame(data)
data_df.head()

,title,timings,latitude,longtitude,price,description,availability_class,avail_status
0,"PVR: Soul Spirit Central Mall, Bellandur\nINFO",09:45 PM,12.9259,77.6754,700.00,RECLINER,_sold,0
1,"PVR: Soul Spirit Central Mall, Bellandur\nINFO",10:30 PM,12.9259,77.6754,700.00,RECLINER,_sold,0
2,Thirumala 4K Dolby Atmos: Agara\nINFO,09:45 PM,12.9199,77.6497,100.00,Silver Class,_available,3
3,Cinephile HSR Layout: PNR Felicity Mall Haralu...,10:00 PM,12.8963,77.6579,200.00,GOLD,_filling,1
4,"PVR: Gold, Nexus (Formerly Forum), Koramangala...",09:30 PM,12.9346,77.6111,1050.00,GOLD PRIME,_sold,0


In [204]:
# # for holding the resultant list
# element_list = []
  
# for page in range(1, 3, 1):
    
#     page_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=" + str(page)
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get(page_url)
#     title = driver.find_elements_by_class_name("title")
#     price = driver.find_elements_by_class_name("price")
#     description = driver.find_elements_by_class_name("description")
#     rating = driver.find_elements_by_class_name("ratings")
  
#     for i in range(len(title)):
#         element_list.append([title[i].text, price[i].text, description[i].text, rating[i].text])
  
# print(element_list)
  
# #closing the driver
# driver.close()